In [49]:
from bs4 import BeautifulSoup
import requests

In [50]:
url = "https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/tsla-20211231.htm"

In [51]:
headers = {
    'User-Agent': 'osslab manager@osslab.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov'  # This is another valid field
}    

In [52]:
response = requests.get(url, headers=headers)

In [53]:
from extractor import extract_content, fast_split_on_tag, split_on_tag

In [54]:
if response.ok:
    soup = BeautifulSoup(response.text, 'html.parser')
    tags = soup.find_all('a', href=True, attrs={'href': '#'})

In [55]:
convert_table_of_contents(soup)

In [78]:
import copy
import re

def test_split_on_tag(soup, block_tag, split_tag, attr=None):
    
    for block in soup.find_all(block_tag):
        splits = block.find_all(split_tag)
        print(len(splits))
        if len(splits) > 0:
            for i, split in enumerate(splits):
                next_split_index = i + 1 if i+1 < len(splits) else None
                next_split = splits[next_split_index] if next_split_index is not None else None

                split_parent = split.parent
                
                if split_tag == "h2":
                    print("split_parent: ", split_parent.name)    
                new_tag = copy.copy(split_parent)
                new_tag.clear()

                index = split_parent.index(split)
                next_index = split_parent.index(next_split) if next_split is not None else None
                print(index, next_index)
                for element in split_parent.contents[index + 1 : next_index]:                         
                    element = element.extract()
                    new_tag.append(element)
                    print("new_tag", new_tag)

                split = split.extract()
                print("extracted split", split)
                split_parent.insert_before(new_tag)
                print("split_parent", split_parent)
                if split_parent == block:
                    if attr is not None:
                        new_tag[attr] = re.sub("\\s+", " ", split.get_text()).strip()
                    #break

                else: split_parent.insert_after(split)



In [100]:
def split_on_tag(soup, block_tag, split_tag, attr=None):
    """Split HTML tree blocks based on split tag."""
    for block in soup.find_all(block_tag):
        splits = (s for s in block.children if s.name == split_tag)
        next_split = None
        for split in splits:
            # Try to get the next split element, but don't consume the generator yet
            #try:
            #    next_split = next(splits)
            #except StopIteration:
            #    break

            split_parent = split.parent
            new_tag = copy.copy(split_parent)
            new_tag.clear()

            index = split_parent.index(split)
            #next_index = split_parent.index(next_split) if next_split is not None else None

            for element in split_parent.contents[index + 1:]:
                print(element)
                if element.name == split_tag:
                    break  
                else:
                    element = element.extract()
                    new_tag.append(element)
                    print("new_tag", new_tag)

            split = split.extract()
            print("extracted split", split)            
            split_parent.insert_after(new_tag)
            print("split_parent", split_parent)
            if split_parent == block and attr is not None:
                new_tag[attr] = re.sub("\\s+", " ", split.get_text()).strip()

            if split_parent != block:
                split_parent.insert_after(split)

In [102]:
html = "<h2>item1</h2><p>test1</p><p>test1</p><p>test1</p><h2>item3</h2><p>test3</p><p>test3</p><p>test3</p><p>test3</p><h2>item4</h2><p>test4</p><p>test4</p><p>test4</p><p>test4</p><p>test4</p><h2>item5</h2><p>test5</p><p>test5</p><p>test5</p><p>test5</p><p>test5</p>"
soup = BeautifulSoup(html, "lxml")
print(soup)
split_on_tag(soup, "html", "h2", "section-h2")
print("final", soup)

<html><body><h2>item1</h2><p>test1</p><p>test1</p><p>test1</p><h2>item3</h2><p>test3</p><p>test3</p><p>test3</p><p>test3</p><h2>item4</h2><p>test4</p><p>test4</p><p>test4</p><p>test4</p><p>test4</p><h2>item5</h2><p>test5</p><p>test5</p><p>test5</p><p>test5</p><p>test5</p></body></html>
final <html><body><h2>item1</h2><p>test1</p><p>test1</p><p>test1</p><h2>item3</h2><p>test3</p><p>test3</p><p>test3</p><p>test3</p><h2>item4</h2><p>test4</p><p>test4</p><p>test4</p><p>test4</p><p>test4</p><h2>item5</h2><p>test5</p><p>test5</p><p>test5</p><p>test5</p><p>test5</p></body></html>
